# Groq

[![Index](https://img.shields.io/badge/Index-blue)](../index.ipynb)
[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/digillia/Digillia-Colab/blob/main/tools/groq.ipynb)

Le code ci-après requiert un compte avec Groq. Groq se distingue par un temps d'inférence extrêmement rapide.
- https://console.groq.com/keys


In [2]:
import os
import sys

# Supprimer les commentaires pour installer (requirements.txt)
# !pip3 install -q -U scikit-learn

# À installer dans tous les cas pour Google Colab et Github
if ('google.colab' in sys.modules) or ('CI' in os.environ):
    !pip3 install -q -U groq
    !pip3 install -q -U groqflow

## Chargement de la Clé pour Groq

Il vous faut obtenir de Groq une clé pour exécuter ce bloc-note Jupyter. Vous pouvez consulter [Getting Started](https://console.groq.com/docs/quickstart). Ensuite, le chargement se fait soit à partir de l'environnement (fichier `.env`), soit à partir des secrets de Google Colab.

In [4]:
if 'google.colab' in sys.modules:
  from google.colab import userdata
  os.environ['GROQ_API_KEY'] = userdata.get('GROQ_API_KEY')
else:
  from dotenv import load_dotenv, find_dotenv
  _ = load_dotenv(find_dotenv()) # lire le fichier .env local

groq_api_key = os.environ['GROQ_API_KEY']

In [5]:
from groq import Groq

client = Groq(api_key=os.environ.get('GROQ_API_KEY'))

## Conversations Chat

In [7]:
MODEL = 'llama3-70b-8192' 
# MODEL = 'mixtral-8x7b-32768'

messages = [
  { 'role': 'system', 'content': 'Tu es un professeur d\'histoire.' },
  { 'role': 'user', 'content': 'Quand la première guerre mondiale a-t-elle commencé?' },
  { 'role': 'assistant', 'content': 'La première guerre mondiale a commencé en 1914.' },
  { 'role': 'user', 'content': 'Quand a-t-elle fini?' }
]

completion = client.chat.completions.create(
    messages=messages,
    model=MODEL  
)

response = completion.choices[0].message.content
print(response)

La Première Guerre mondiale a pris fin le 11 novembre 1918, lorsque l'armistice a été signé entre les Alliés et l'Allemagne. Cependant, le traité de Versailles, qui a officiellement mis fin à la guerre, n'a été signé que le 28 juin 1919.


## Groqflow

In [25]:
import numpy as np
# from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

random_state=123

# Créer les données
# X, y = make_classification(
#    n_samples=500,
#    n_features=2,
#    n_informative=2,
#    n_redundant=0,
#    n_repeated=0,
#    n_classes=2,
#    n_clusters_per_class=2,
#    class_sep=.9,
#    random_state=random_state,
# )

# Créer les données
np.random.seed(random_state)
X = np.random.rand(1000, 10).astype(np.float32)
y = np.random.randint(2, size=1000)

# Diviser les données en ensemble d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_state)

# Entraîner le modèle
clf = RandomForestClassifier(max_depth=3, random_state=random_state)
clf.fit(X_train, y_train)

# Rapport
print(classification_report(y_test, clf.predict(X_test)))

In [27]:
from groqflow import groqit

# Construire un modèle pour Groq
gq_clf = groqit(clf, {'input_0': X_test})

# Rapport
print(classification_report(y_test, gq_clf.predict(X_test)))

IntakeError: Argument 'model' passed to build_model() is of unsupported type <class 'sklearn.ensemble._forest.RandomForestClassifier'>